# MIMIC

## Imports and pointers

First, begin with imports and data pointers

In [11]:
from Bio.PDB.PDBParser import PDBParser
from Bio.Seq import Seq
# from Bio import motifs, SeqIO
from Bio import motifs
from pathlib import Path
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn import metrics
from ipywidgets import interact
# from sklearn.datasets import make_blobs
# from sklearn.preprocessing import StandardScaler
# from scipy.cluster import hierarchy
import numpy as np

import Levenshtein as lev

parser = PDBParser(PERMISSIVE=1)

structure_id = "pepsurf"
pdb_file = Path("data/pdb/6vxx_pepsurf_1592103509_pipe.pdb")
mast_file = Path("data/meme-suite/mast.xml")
full_file = Path("data/full.txt")

## Inspect the PDB file

In [12]:
structure = parser.get_structure(structure_id, pdb_file)
print("Resolution:", structure.header["resolution"])
print("Keywords:", structure.header["keywords"])

Resolution: 2.8
Keywords: coronavirus, sars-cov-2, sars-cov, spike glycoprotein, fusion protein, structural genomics, seattle structural genomics center for infectious disease, ssgcid, viral protein
C:\Users\Sasha\Anaconda3\envs\biopython\lib\site-packages\Bio\PDB\StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 25539.
  PDBConstructionWarning,
C:\Users\Sasha\Anaconda3\envs\biopython\lib\site-packages\Bio\PDB\StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 25833.
  PDBConstructionWarning,
C:\Users\Sasha\Anaconda3\envs\biopython\lib\site-packages\Bio\PDB\StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 26127.
  PDBConstructionWarning,


## Inspect the MAST data

In [13]:
with open(mast_file, 'r') as handle:
    record = motifs.parse(handle, "mast")

# meme_file = Path("data/meme-suite/meme.xml")
# with open(meme_file) as handle:
#     record = motifs.parse(handle, "meme")
print(record.alphabet)
print(len(record.sequences))
print(record.version)
print("Available Attributes:")
print(record[0].__dict__)
 
for motif in record[:3]:
    print(motif.name, motif.id)

mast_data = []
for motif in record:
    mast_data.append(motif.id)

print(mast_data[:3])
print(len(mast_data)) 
print(len(mast_data) == len(set(mast_data)))
print(len(set(mast_data)))

Protein
2534
5.1.1
Available Attributes:
{'name': '1', 'counts': None, 'instances': None, 'length': 8, 'alphabet': 'Protein', '_pseudocounts': {'P': 0.0, 'r': 0.0, 'o': 0.0, 't': 0.0, 'e': 0.0, 'i': 0.0, 'n': 0.0}, '_background': {'P': 0.14285714285714285, 'r': 0.14285714285714285, 'o': 0.14285714285714285, 't': 0.14285714285714285, 'e': 0.14285714285714285, 'i': 0.14285714285714285, 'n': 0.14285714285714285}, '_Motif__mask': (), 'evalue': 0.0, 'num_occurrences': 0, 'id': 'ADVDLISM', 'alt_id': 'MEME-1'}
1 ADVDLISM
2 GQYHVNEM
3 EVQDRVD
['ADVDLISM', 'GQYHVNEM', 'EVQDRVD']
900
False
478


## Inspect the Excel data (copied to a text file)

In [14]:
# Get the raw data as a list and inspect
data = []
with open(full_file, 'r') as handle:
    for count, line in enumerate(handle, start=1):
        if count % 2 == 0:
            data.append(line.rstrip())
            
print('Length of the data:', len(data))
print('Unique entries in the data:', len(set(data)))
print('Number of entries different from the mast data:', len(set(data).difference(set(mast_data))))

# Generate the full distance matrix
n_samples = len(data)
dist_matrix = np.zeros((n_samples, n_samples))
for i in range(n_samples):
    for j in range(n_samples):
        dist_matrix[i, j] = lev.distance(data[i], data[j])

print('Raw Data Distance Matrix:')
print(dist_matrix)
print('Maximum distance within the matrix:', max(dist_matrix[0]))

Length of the data: 562
Unique entries in the data: 562
Number of entries different from the mast data: 506
Raw Data Distance Matrix:
[[0. 6. 7. ... 7. 6. 8.]
 [6. 0. 6. ... 8. 7. 7.]
 [7. 6. 0. ... 6. 7. 8.]
 ...
 [7. 8. 6. ... 0. 3. 4.]
 [6. 7. 7. ... 3. 0. 3.]
 [8. 7. 8. ... 4. 3. 0.]]
Maximum distance within the matrix: 8.0


In [15]:
import scipy.spatial.distance as ssd
from scipy.cluster import hierarchy as h
import matplotlib.pyplot as plt

distArray = ssd.squareform(dist_matrix)
Z = h.ward(distArray)

@interact
def get_dendrogram(truncate=(0, 10, 1), threshold=(0, 40, 1)):
    # convert the redundant n*n square matrix form into a condensed nC2 array
    plt.figure(figsize=(12, 6))
    plt.title('Naive Hierarchy')
    h.dendrogram(Z,truncate_mode='level', p=truncate, leaf_rotation=0, orientation="left", color_threshold=threshold)
    plt.axvline(x=threshold, linewidth=4, color='r')
    plt.xlabel("Number of points in node (or index of point if no parenthesis)")
    plt.ylabel("Bins")
    plt.show()

interactive(children=(IntSlider(value=5, description='truncate', max=10), IntSlider(value=20, description='thr…

In [16]:
from scipy.cluster.hierarchy import dendrogram

def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

# setting distance_threshold=0 ensures we compute the full tree.
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None).fit(dist_matrix)

@interact
def gen_diagram(truncate=(0, 10, 1), threshold=(0, 700, 1)):
    plt.figure(figsize=(12, 6))
    plt.title('Agglomerative Hierarchy, P=' + str(threshold))
    # plot the top three levels of the dendrogram
    plot_dendrogram(model, truncate_mode='level', p=truncate, leaf_rotation=0, orientation="left", color_threshold=threshold, above_threshold_color='black')
    # plot_dendrogram(model, truncate_mode='level', p=truncate, color_threshold=threshold)
    # plt.yscale('log', basey=2)
    plt.axvline(x=threshold, linewidth=4, color='r')
    plt.ylabel("Number of points in node (or index of point if no parenthesis)")
    plt.xlabel("Bins")
    plt.show()

C:\Users\Sasha\Anaconda3\envs\biopython\lib\site-packages\scipy\cluster\hierarchy.py:826: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  return linkage(y, method='ward', metric='euclidean')


interactive(children=(IntSlider(value=5, description='truncate', max=10), IntSlider(value=350, description='th…

In [17]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

# #########################################################################
# Compute DBSCAN
@interact
def get_db(eps=10, min_samples=(0,40,1)):
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(dist_matrix)
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_

    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)

    print('Estimated number of clusters: %d' % n_clusters_)
    print('Estimated number of noise points: %d' % n_noise_)

# agc = AgglomerativeClustering().fit(dist_matrix)
# agc.labels_

interactive(children=(IntSlider(value=10, description='eps', max=30, min=-10), IntSlider(value=20, description…

In [18]:
# print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
# print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
# print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
# print("Adjusted Rand Index: %0.3f"
#       % metrics.adjusted_rand_score(labels_true, labels))
# print("Adjusted Mutual Information: %0.3f"
#       % metrics.adjusted_mutual_info_score(labels_true, labels))
# print("Silhouette Coefficient: %0.3f"
#       % metrics.silhouette_score(X, labels))

# # #############################################################################
# # Plot result
# import matplotlib.pyplot as plt

# # Black removed and is used for noise instead.
# unique_labels = set(labels)
# colors = [plt.cm.Spectral(each)
#           for each in np.linspace(0, 1, len(unique_labels))]
# for k, col in zip(unique_labels, colors):
#     if k == -1:
#         # Black used for noise.
#         col = [0, 0, 0, 1]

#     class_member_mask = (labels == k)

#     xy = X[class_member_mask & core_samples_mask]
#     plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
#              markeredgecolor='k', markersize=14)

#     xy = X[class_member_mask & ~core_samples_mask]
#     plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
#              markeredgecolor='k', markersize=6)

# plt.title('Estimated number of clusters: %d' % n_clusters_)
# plt.show()